In [1]:
import pickle
import tensorflow as tf
import datetime
import json
import requests

from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [3]:
user_id_enc = pickle.load(open('user_id_enc.pkl', 'rb'))
movie_id_enc = pickle.load(open('movie_id_enc.pkl', 'rb'))

user_id_enc, movie_id_enc

(LabelEncoder(), LabelEncoder())

In [4]:
model = tf.keras.models.load_model('neural_model')

In [5]:
u = 15
q = 'star'

In [6]:
 query = {
        'query': {
            'match': {
                'title': q
            }
        },
        'sort': [{'_score': 'desc'}]
    }
res = es.search(index='movies', doc_type='movie', body=json.dumps(query))
hits = res['hits']['hits']
hits

[{'_index': 'movies',
  '_type': 'movie',
  '_id': '800',
  '_score': 6.2658124,
  '_source': {'title': 'Lone Star (1996)',
   'genres': ['Drama', 'Mystery', 'Western']}},
 {'_index': 'movies',
  '_type': 'movie',
  '_id': '1613',
  '_score': 6.2658124,
  '_source': {'title': 'Star Maps (1997)', 'genres': ['Drama']}},
 {'_index': 'movies',
  '_type': 'movie',
  '_id': '1750',
  '_score': 6.2658124,
  '_source': {'title': 'Star Kid (1997)',
   'genres': ['Adventure', 'Children', 'Fantasy', 'Sci-Fi']}},
 {'_index': 'movies',
  '_type': 'movie',
  '_id': '4757',
  '_score': 6.2658124,
  '_source': {'title': 'Rock Star (2001)',
   'genres': ['Comedy', 'Drama', 'Musical']}},
 {'_index': 'movies',
  '_type': 'movie',
  '_id': '26487',
  '_score': 6.2658124,
  '_source': {'title': 'Star 80 (1983)', 'genres': ['Drama']}},
 {'_index': 'movies',
  '_type': 'movie',
  '_id': '68358',
  '_score': 6.2658124,
  '_source': {'title': 'Star Trek (2009)',
   'genres': ['Action', 'Adventure', 'Sci-Fi', '

In [8]:
# movieId = _id
# ratingScore = _score
max_score = max(x['_score'] for x in hits)
examples = [x['_id'] for x in hits]

user_id_encoded = user_id_enc.transform([u])
def get_rating(movie_id):
    model_input = {'movie_id': movie_id_enc.transform([int(movie_id)]), 'user_id': user_id_encoded}
    return model.predict(model_input)[0][0][0]

ratings = [get_rating(x) for x in examples]
for h, r in zip(hits, ratings):
    h['_r'] = r

def rating_rank_boost(rating):
    return rating / max_score

results = sorted(hits, key=lambda x: 0.6 * x['_score'] + 0.4 * rating_rank_boost(x['_r']), reverse=True)
results

[{'_index': 'movies',
  '_type': 'movie',
  '_id': '800',
  '_score': 6.2658124,
  '_source': {'title': 'Lone Star (1996)',
   'genres': ['Drama', 'Mystery', 'Western']},
  '_r': 3.7716784},
 {'_index': 'movies',
  '_type': 'movie',
  '_id': '68358',
  '_score': 6.2658124,
  '_source': {'title': 'Star Trek (2009)',
   'genres': ['Action', 'Adventure', 'Sci-Fi', 'IMAX']},
  '_r': 3.4773805},
 {'_index': 'movies',
  '_type': 'movie',
  '_id': '1613',
  '_score': 6.2658124,
  '_source': {'title': 'Star Maps (1997)', 'genres': ['Drama']},
  '_r': 2.0342047},
 {'_index': 'movies',
  '_type': 'movie',
  '_id': '26487',
  '_score': 6.2658124,
  '_source': {'title': 'Star 80 (1983)', 'genres': ['Drama']},
  '_r': 1.5647167},
 {'_index': 'movies',
  '_type': 'movie',
  '_id': '1750',
  '_score': 6.2658124,
  '_source': {'title': 'Star Kid (1997)',
   'genres': ['Adventure', 'Children', 'Fantasy', 'Sci-Fi']},
  '_r': 1.3835651},
 {'_index': 'movies',
  '_type': 'movie',
  '_id': '4757',
  '_scor